### Pattern finding functions

In [1]:
def find_pattern_first_index(bitstream):
    pattern = "3c5c"
    pattern_length = len(pattern) * 4  # 4 bits per hexadecimal digit

    for i in range(len(bitstream) - pattern_length + 1):
        current_substring = bitstream[i:i+pattern_length]
        hexadecimal = hex(int(current_substring, 2))[2:]  # Convert binary to hexadecimal
        if hexadecimal == pattern:
            return i

    return -1  # Pattern not found

def find_pattern_multiple_indexs(bitstream):
    pattern = "3c5c"
    pattern_length = len(pattern) * 4  # 4 bits per hexadecimal digit

    indices = []  # List to store the indices of pattern occurrences

    for i in range(len(bitstream) - pattern_length + 1):
        current_substring = bitstream[i:i+pattern_length]
        hexadecimal = hex(int(current_substring, 2))[2:]  # Convert binary to hexadecimal
        if hexadecimal == pattern:
            indices.append(i)

    return indices  # Return the list of indices

### Translate binary to readable data

In [2]:
def translate(input_stream, parent_dir, output, chipID):
    f = open(parent_dir+'/'+output, 'w')
    i=0
    residual = ''
    # hex chipID to binary
    binID = format(int(chipID, 0), '017b')
    wordlength = 40
    while True:
        word = input_stream[wordlength*i: wordlength*(i+1)]
        printline = "ETROC2 0 "# + "{:d} ".format(channel)
        if len(word) != 40:
            # print('Cannot form a 40 bit word')
            i += 1
            residual = word
            break
        elif len(word) == 40:
            # Header
            if word[0:16] == '0011110001011100' and word[16:18] == '00':
                printline += "HEADER "
                printline += "L1COUNTER " + word[18:26] + " "
                printline += "TYPE " + word[26:28] + " "
                printline += "BCID " + f"{int(word[28:40], base=2)}"
                i += 1
            # Trailer
            elif word[0:18] == '0'+str(binID):
                printline += "TRAILER "
                printline += "CHIPID " + f"{hex(int(word[1:18], base=2))}" + " "
                printline += "STATUS " + word[18:24] + " "
                printline += "HITS " + f"{int(word[24:32], base=2)}" + " "
                printline += "CRC " + word[32:40]
                i += 1
            # Frame filler
            elif word[0:16] == '0011110001011100' and word[16:18] == '10':
                printline += "FRAMEFILLER "
                printline += "L1COUNTER " + word[18:26] + " "
                printline += "EBS " + word[26:28] + " "
                printline += "BCID " + f"{int(word[28:40], base=2)}"
                i += 1
            # Firmware filler
            elif word[0:16] == '0011110001011100' and word[16:18] == '11':
                printline += "FIRMWAREFILLER "
                printline += "MISSINGCOUNT " + word[18:40]
                i += 1
            # DATA
            elif word[0] == '1':
                printline += "DATA "
                printline += "EA " + word[1:3] + " "
                printline += "COL " + "{:d} ".format(int(word[3:7], base=2))
                printline += "ROW " + "{:d} ".format(int(word[7:11], base=2))
                printline += "TOA " + "{:d} ".format(int(word[11:21], base=2))
                printline += "TOT " + "{:d} ".format(int(word[21:30], base=2))
                printline += "CAL " + "{:d} ".format(int(word[30:40], base=2))
                i += 1
            else:
                printline += "NOT DEFINED " + word[0:16] + " " + word[16:18] + " " + word[18:]
                i += 1
                pass
        
        f.write(printline+'\n')
    f.close()
    
    return residual

In [5]:
from glob import glob
from natsort import natsorted, ns
from tqdm import tqdm

# natural sorting in python
files = glob('test3-third/*.dat')
files = natsorted(files)

In [6]:
residual = ''

for i, ifile in enumerate(tqdm(files)):
    # Let's make a very long bitstream single line
    bitstream = residual + ''
    with open(ifile, 'r') as infile:
        # make data bitstream in a single line
        for line in infile.readlines():
            if line[0:4] == '1100':
                bitstream += line.strip()[4:]
    
    # Find the first index of 3c5c pattern
    position = find_pattern_first_index(bitstream)
    out_bitstream = bitstream[position:]

    outname = 'TDC_Data_translated_'+str(i)+'.dat'
    residual = translate(out_bitstream, 'test3-third', outname, '0x17f0f')

100%|██████████| 48/48 [00:04<00:00,  9.64it/s]
